**Resumes**

Sorry for downloading your resume. I need it for testing

In [ ]:
resume = "./resumes/cw-resume-jv1.pdf"

In [ ]:
resume = "./resumes/Jerick-Iquin.pdf"

In [ ]:
resume = "./resumes/Kenneth Lee Resume.pdf"

In [ ]:
resume = "./resumes/Justin Lam - Resume - vPortfolio.pdf"

**Using pdfminer.six**

- Probably will use. Best so far

In [ ]:
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LAParams

laparams = LAParams(line_margin=1)

In [ ]:
# extract pages outputs an iterable so we can create list from it
pages = [x for x in extract_pages(resume)]

for page in range(len(pages)):
    print('page_number', page)
    text = extract_text(resume, page_numbers=page, laparams=laparams)
    print(text)

**Testing Beautiful Soup**

This is to do webscraping of your linkedin profile

- Linkedin detects automated scraping and blocks requests
- Need to install chrome and chromedriver

In [261]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def scrape_linkedin_profile(url):
    options = Options()
    # running in headless mode
    options.add_argument('--headless') 
    driver = webdriver.Chrome(executable_path='chromedriver', options=options)

    driver.get(url)

    # wait for the page to load. adjust sleep here
    driver.implicitly_wait(5)

    html_content = driver.page_source

    soup = BeautifulSoup(html_content, 'html.parser')

    lis = soup.find_all('li', class_='experience-item')

    modal_dismis = driver.find_element(By.CLASS_NAME, 'contextual-sign-in-modal__modal-dismiss')
    modal_dismis.click()


    show_more_buttons = driver.find_elements(By.CLASS_NAME, 'show-more-less-text__button--more')

    for button in show_more_buttons:
        button.click()

    for li in lis:
        print("Experience", lis.index(li))
        description_more = ''
        description_less = ''
        
        title = get_text(li.find('h3', class_='profile-section-card__title'))
        company = get_text(li.find('h4', class_='profile-section-card__subtitle'))
        location = get_text(li.find('p', class_='experience-item__location'))
        description_more = get_text(li.find('p', class_='show-more-less-text__text--more'))
        description_less = get_text(li.find('p', class_='show-more-less-text__text--less'))

        if description_more:
            description_list = get_duties(description_more)
        elif description_less:
            description_list = get_duties(description_less)
        else:
            description_list = []
            
        print(title)
        print(company)
        print(location)
        print('Responsibilities: ')
        for duty in description_list:
            print(duty)
        print('\n')
        
    driver.quit()

def get_text(element):
    try:
        return element.get_text().strip()
    except Exception as e:
        return ''
    
def get_duties(description):
    trimmed = description.replace('\n            \n\n    \n    \n\n    \n        Show less', '')
    return trimmed.split('• ')[1:]


In [262]:
scrape_linkedin_profile('https://www.linkedin.com/in/jerick-iquin/')

Experience 0
Full Stack Developer
BalloonBox
Vancouver, British Columbia, Canada
Responsibilities: 
Architected and led front-end development of a cutting-edge web3 product that launched successfully under a tight deadline, driving our client’s total staked token amount to soar above US$800K
Spearheaded the development of a game-changing extension on top of our client’s existing product architecture, enabling them to demo and market their offerings in n-fold ways
Led front-end development of a B2B platform that allows natural resource consumers (manufacturers) to offset their carbon impact by virtually giving back to communities affected by natural resource extraction
Boosted user approval rate for our client’s digital wallet by triple-digit percentage points by implementing Plaid and OAuth for wallet authentication
Reduced project setup up by 75% by building front end code base template and carrying out best practices through strict linting rules
Mentored junior colleagues through ext

**Using pypdf package**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from pypdf import PdfReader

reader = PdfReader(resume)
number_of_pages = len(reader.pages)

for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)

**Using pyPDF2**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader(resume, strict=True)
number_of_pages = len(reader.pages)
for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)
